### Quick Description

This notebook calculates the toxicity score from each sentence in a subtitles' file. Finally, it persists the data in a DataFrame like output.

In [2]:
import yaml
import glob
import os
import pandas as pd
import time
import re
import logging
import sys

from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sys.path.append("../utils")
from toxicity_api_communication import get_toxicity_score

In [3]:
filepaths = yaml.load(open("../config/filepaths.yaml"))
credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]

/tmp/ipykernel_65392/1089545284.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("../config/filepaths.yaml"))
/tmp/ipykernel_65392/1089545284.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]


In [4]:
def calculate_toxicity_scores(credentials, input_path, output_path):
    analyzer = SentimentIntensityAnalyzer()

    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]

    for filename in tqdm(filenames, total=len(filenames)):
        df_sentences = pd.read_csv(os.path.join(input_path, filename))
        for i, row in df_sentences.iterrows():
            df_sentences.loc[i, "score"] = analyzer.polarity_scores(row.text)["neg"]

        df_sentences.to_csv(os.path.join(output_path, filename), index=False)

In [5]:
calculate_toxicity_scores(
    credentials,
    filepaths["model_based_segmentated_data"],
    filepaths["vader_scored_data"]
)

100%|███████████████████████████████████████████████████████████████| 552/552 [00:06<00:00, 81.20it/s]
